In [ ]:

from pathlib import Path
from mllm.markov_games.rollout_tree import *
from mllm.markov_games.gather_and_export_utils import (
     gather_simulation_stats, 
     get_rollout_trees, get_rollout_tree_paths, gather_all_rewards, gather_all_chat_turns_for_path, get_all_agents
)
from mllm.markov_games.trust_and_split.tas_simulation import SplitsLog
import matplotlib.pyplot as plt
import numpy as np
from statistics import mean
from collections import defaultdict
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

In [ ]:
# Get custom path from user input
custom_path = input("Enter the path to your rollout data: ").strip()

rollout_trees = get_rollout_trees(custom_path)
print(f"Rollout trees from {len(rollout_trees)} iterations loaded from: {custom_path}")

In [ ]:
main_rollouts_paths = [[get_rollout_tree_paths(root)[0] for root in roots] for roots in rollout_trees]

In [ ]:
def greedy_level(agent_id: str, sl: SimulationStepLog): 
    return sl.info["coins_given_to_self"][agent_id] / sl.info["num_coins"]

def split_filter(sl: SimulationStepLog):
    info : dict = sl.info
    if "coins_given_to_self" in info and "num_coins" in info:
        return True
    return False
nb_iterations = len(rollout_trees)
agent_ids = get_all_agents(rollout_trees[0][0])
print(agent_ids)
agent_greedy_levels = {}
for agent_id in agent_ids:
    stat_func = lambda sl: greedy_level(agent_id, sl)
    greedy_levels = [[gather_simulation_stats(path=path, filter=split_filter, stat_func=stat_func) for path in paths] for paths in main_rollouts_paths]
    greedy_levels = [np.mean(np.array(gls)) for gls in greedy_levels]
    plt.plot(np.arange(len(greedy_levels)), greedy_levels, label=agent_id)
import matplotlib.ticker as mtick
plt.style.use('plotstyle.mplstyle')
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(1))
plt.xticks(range(0, nb_iterations))
plt.xlabel("Iteration")
plt.ylabel("Average Percentage of Coins Given to Self")
plt.legend()
plt.show()

In [ ]:
plt.cla()
